Step 1: Download data files 
Preferred script: download_data_ooi1_0.py

Usage:
1- compare the QC Database to the OOI GUI data catalog, 
2- build netCDF data request urls, and 
3- send requests if prompted.
To Run:
Input variables:
sDir: directory where outputs are saved
username: OOI API username
Token: OOI API password

Module Functions: in Tools (GitHub)
Download_data_nc_interactive.py (Main Script)  
data_request_urls
send_data_requests_nc
data_request_tools not in use 
Interactive_inputs
Prompt usage:
select the platform, node, instruments and methods
start downloading data

In [1]:
import datetime as dt
import pandas as pd
import os
import itertools
import functions.common as cf
import scripts

In [2]:
sDir = '/Users/leila/Documents/Desktop/8thEGOMeeting-notebooks'
f = ''  # optional i.e. 'data_download.csv'
username = 'OOIAPI-4J3Y5ISGDUPDB7'
token = '4OT8YQUME0XOQF'

cf.create_dir(sDir)
now = dt.datetime.now().strftime('%Y%m%dT%H%M')

In [5]:
if not f:
    array, subsite, node, inst, delivery_methods = scripts.interactive_inputs.return_interactive_inputs()
    f_url_list = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)

AttributeError: module 'scripts' has no attribute 'interactive_inputs'

In [ ]:
if not f:
    array, subsite, node, inst, delivery_methods = scripts.interactive_inputs.return_interactive_inputs()
    f_url_list = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
else:
    df = pd.read_csv(os.path.join(sDir, f))
    url_list = []
    for i, j in df.iterrows():
        array = scripts.data_request_tools.check_str(j['array'])
        array = scripts.data_request_tools.format_inputs(array)
        refdes = j['reference_designator']
        if type(refdes) == str:
            subsite = scripts.data_request_tools.format_inputs(refdes.split('-')[0])
            node = scripts.data_request_tools.format_inputs(refdes.split('-')[1])
            inst = scripts.data_request_tools.format_inputs('-'.join((refdes.split('-')[2], refdes.split('-')[3])))
        else:
            subsite = scripts.data_request_tools.check_str(j['subsite'])
            subsite = scripts.data_request_tools.format_inputs(subsite)
            node = scripts.data_request_tools.check_str(j['node'])
            node = scripts.data_request_tools.format_inputs(node)
            inst = scripts.data_request_tools.check_str(j['sensor'])
            inst = scripts.data_request_tools.format_inputs(inst)
        delivery_methods = scripts.data_request_tools.check_str(j['delivery_method'])
        delivery_methods = scripts.data_request_tools.format_inputs(delivery_methods)

        urls = scripts.data_request_urls_ooi1_0.main(sDir, array, subsite, node, inst, delivery_methods, now)
        url_list.append(urls)

    f_url_list = list(itertools.chain(*url_list))

thredds_output_urls = scripts.send_data_requests_nc.main(sDir, f_url_list, username, token, now)

print('\nSeeing if the requests have fulfilled...')
for i in range(len(thredds_output_urls)):
    url = thredds_output_urls[i]
    print('\nDataset {} of {}: {}'.format((i + 1), len(thredds_output_urls), url))
    if 'no_output_url' not in url:
        cf.check_request_status(url)